In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torchvision.transforms as transforms

In [8]:
class autoencoder(nn.Module):
  def __init__(self, x_dim, hidden_dim, z_dim=10):
    super(autoencoder,self).__init__()
    self.enc_layer1 = nn.Linear(x_dim, hidden_dim)
    self.enc_layer2 = nn.Linear(hidden_dim, z_dim)

    self.dec_layer1 = nn.Linear(z_dim, hidden_dim)
    self.dec_layer2 = nn.Linear(hidden_dim, x_dim)

  def encode(self, x):
    x = F.relu(self.enc_layer1(x))
    z = F.relu(self.enc_layer2(x))
    return z

  def decode(self, z):
    output = F.relu(self.dec_layer1(z))
    output = F.relu(self.dec_layer2(output))
    return output

  def forward(self, x):
    z = self.encode(x)
    output = self.decode(z)
    return output 




In [9]:
autoencoder = autoencoder(256,32)